## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [70]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 9 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [71]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [72]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [73]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210114.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [74]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,209520,6800,216320,123459,0.570724,2021-01-13
1,Aragón,35415,1200,36615,20854,0.569548,2021-01-13
2,Asturias,35420,1200,36620,29074,0.793938,2021-01-14
3,Baleares,12840,600,13440,7552,0.561905,2021-01-13
4,Canarias,41310,1300,42610,25159,0.590448,2022-01-13
5,Cantabria,17700,500,18200,12779,0.702143,2021-01-14
6,Castilla y Leon,76740,2500,79240,57969,0.731562,2021-01-14
7,Castilla La Mancha,53100,1700,54800,30613,0.558631,2021-01-14
8,Cataluña,180020,5800,185820,105587,0.568222,2021-01-13
9,C. Valenciana,91450,2900,94350,80429,0.852454,2021-01-14


..hacemos una copia para hacerle las transformaciones necesarias:

In [84]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [85]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [86]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [87]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [88]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [89]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [90]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [91]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [92]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Fecha de la ultima vacuna registrada,source_name,source
0,14/1/21,Andalucía,209520,6800,216320,123459,0.570724,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,14/1/21,Aragón,35415,1200,36615,20854,0.569548,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,14/1/21,Asturias,35420,1200,36620,29074,0.793938,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,14/1/21,Baleares,12840,600,13440,7552,0.561905,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,14/1/21,Canarias,41310,1300,42610,25159,0.590448,13/1/22,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,14/1/21,Cantabria,17700,500,18200,12779,0.702143,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,14/1/21,Castilla y Leon,76740,2500,79240,57969,0.731562,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,14/1/21,Castilla La Mancha,53100,1700,54800,30613,0.558631,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,14/1/21,Cataluña,180020,5800,185820,105587,0.568222,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,14/1/21,C. Valenciana,91450,2900,94350,80429,0.852454,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [93]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [117]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [118]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
135,13/1/21,Navarra,17700,7420,0.419209,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
136,13/1/21,País Vasco,47205,17771,0.376464,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
137,13/1/21,Ceuta,1005,820,0.815920,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
138,13/1/21,Melilla,1005,1005,1.000000,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [119]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [120]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source',
       'Dosis entregadas Pfizer', 'Dosis entregadas Moderna'],
      dtype='object')

In [121]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Última fecha de actualización de datos','Fecha de la ultima vacuna registrada',
                                     'source_name', 'source']]

In [126]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...
155,14/1/21,Navarra,17700.0,500.0,18200,8314,0.456813,NaN,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
156,14/1/21,País Vasco,47205.0,1400.0,48605,20003,0.411542,NaN,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
157,14/1/21,Ceuta,1005.0,200.0,1205,911,0.756017,NaN,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
158,14/1/21,Melilla,1005.0,200.0,1205,1098,0.911203,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [125]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [127]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)